In [1]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print("✅ Success! Your M4 GPU (MPS) is active.")
else:
    print("❌ MPS not found. using CPU.")

✅ Success! Your M4 GPU (MPS) is active.


In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets

# 1. Setup Device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# 2. Data Preparation
# Resize images to 224x224 (ResNet standard) and normalize
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Assume you have a folder structure: data/train/plastic, data/train/paper, etc.
# train_dataset = datasets.ImageFolder(root='data/train', transform=transform)
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)

# 3. Load Pre-trained ResNet
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# 4. Modify the Final Layer
# Freeze early layers so we don't destroy pre-trained knowledge
for param in model.parameters():
    param.requires_grad = False

# Replace the last layer (fc) to match your number of waste classes (e.g., 4 classes)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4) 

model = model.to(device) # Move model to M4 GPU

print(f"Model ready on {device}. Ready to train!")

Model ready on mps. Ready to train!


In [10]:
from ultralytics import YOLO

# 1. Load the model
model = YOLO('yolov8n.pt') 

# 2. Train on the tiny COCO8 dataset
# This should take less than 10-20 seconds total.
results = model.train(
    data='coco8.yaml',   # <--- The 8-image dataset
    epochs=10,           # We only need a few epochs to test speed
    imgsz=640, 
    device='mps',        # Use your M4 GPU
    batch=4,             # Small batch for small data
    workers=0            # Fix for the "Data Ping-Pong" issue
)

New https://pypi.org/project/ultralytics/8.3.249 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.248 🚀 Python-3.12.12 torch-2.5.1 MPS (Apple M4)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco8.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overl

In [12]:
import segmentation_models_pytorch as smp
import torch

device = torch.device("mps")

# Create U-Net with a ResNet encoder backbone
model = smp.Unet(
    encoder_name="resnet34",        # encoder backbone
    encoder_weights="imagenet",     # use pre-trained weights
    in_channels=3,                  # input channels (RGB)
    classes=1                       # output class (waste vs background)
)

model.to(device)

# Example Input (Batch size 4, 3 channels, 256x256 size)
# Note: Segmentation images can be heavy. 256x256 is a good start on 16GB RAM.
input_tensor = torch.rand(4, 3, 256, 256).to(device)
output = model(input_tensor)

print(f"Output shape: {output.shape}") # Should be [4, 1, 256, 256]

/opt/miniconda3/envs/torch312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Output shape: torch.Size([4, 1, 256, 256])


In [2]:
from ultralytics import YOLO

# 1. Load the model you just trained
# Note: We use the path where your notebook said it saved results
model = YOLO('runs/detect/train/weights/best.pt') 

# 2. Run live inference on the webcam (source='0')
# It will open a window showing the video feed. Press 'q' to quit.
model.predict(source='0', show=True, conf=0.5)


1/1: 0... Success ✅ (inf frames of shape 1920x1080 at 15.00 FPS)

WARNING ⚠️ 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

0: 384x640 (no detections), 34.6ms
0: 384x640 1 person, 31.0ms
0: 384x640 1 person, 33.0ms
0: 384x640 1 person, 34.7ms
0: 384x640 1 person, 33.9ms
0: 384x640 1 person, 30.9ms
0: 384x640 1 person, 32.5ms
0: 384x640 1 person, 33.0ms
0: 384x640 1 person, 32.4ms
0: 384x640 1 person, 31.4ms
0: 384x640 1 person, 30.6ms
0: 384x640 1 person, 35.1ms
0: 384x640 1 person, 34.0ms
0: 384x640 1 person, 

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted p